# AI-Powered Marketing Brochures
## Business Problem

Create a product that can gernate marketing brochures about a company in multiple languages
- For prospective clients
- For investors
- For recruitment

The technology
- Use OpenAI API
- Use one-shot prompting (with one example) then add multi-shot to refine it
- Stream back results and show with formatting

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import anthropic

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

claude = anthropic.Anthropic()

API key looks good so far
Anthropic API Key exists and begins sk-ant-


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## Link prompts
### Multi-shot system prompt

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in these examples:"
link_system_prompt += """
Example 1
['https://my-company.com', 'https://my-company.com/about-me', 'https://www.linkedin.com/in/my-company/', 'mailto:joe.blog@gmail.com', 'https://my-company.com/news', '/case-studies', 'https://patents.google.com/patent/US20210049536A1/', 'https://my-company.com/workshop-ai']

    Links:
{
    "links": [
        {"type": "landing page", "url": "https://great-comps.com/about-me"},
        {"type": "about page", "url": "https://great-comps.com/about-me"},
        {"type": "news page": "url": "https://great-comps.com/news"},
        {"type": "case studies page": "url": "https://great-comps.com/case-studies"},
        {"type": "workshop page": "url": "https://great-comps.com/workshop-ai"},
    ]
}
Example 2
['https://www.acmeinc.com', '/#about', '/#projects', '/#experience', '/#skills', 'https://github.com/acmeinc']

    Links:
{
    "links": [
        {"type": "landing page", "url": "https://www.acmeinc.com"},
        {"type": "GitHub projects": "url": "https://github.com/acmeinc"},
    ]
}
"""

In [ ]:
print(link_system_prompt)

### User prompt

## Get links

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

## Create brochure

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

### System prompt

In [8]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


### User prompt

#### Generate brochure in English

In [9]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [10]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [13]:
brochure_text = create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'landing page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
**The AI Community Building the Future**  
Hugging Face is a dynamic platform at the forefront of the machine learning community, fostering collaboration on models, datasets, and innovative applications. Join us to explore over **1 million models** and engage with a passionate community dedicated to accelerating AI development.

---

## What We Offer

### Collaborate & Innovate
Hugging Face is the go-to platform for machine learning enthusiasts and professionals alike. 
- **Models**: Browse through a vast collection of the latest AI models.
- **Datasets**: Access over **250,000 datasets** tailored for various ML tasks.
- **Spaces**: Create and share applications effortlessly.

### Cutting-edge Technology
Utilize our **HF Open Source stack** to enhance your projects across text, image, video, audio, and even 3D modalities. Take advantage of our state-of-the-art libraries like:
- **Transformers:** Advanced ML for PyTorch, TensorFlow, and JAX.
- **Diffusers:** Leading diffusion models.
- **Tokenizers:** Fast tokenizers optimized for research and production.

---

## Our Customers
With over **50,000 organizations** using our platform, we empower both small startups and Fortune 500 companies, including:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

Our solutions cater to enterprises seeking advanced platforms with robust security and dedicated support, starting at **$20/user/month**.

---

## Company Culture
At Hugging Face, we value collaboration, openness, and innovation. Our inclusive culture fosters creativity and encourages team members to contribute and share their ideas. We believe in building a supportive environment where everyone can learn and grow together, embedded in the vibrant AI community.

---

## Careers at Hugging Face
We are always on the lookout for talented individuals to join our team. If you are passionate about machine learning, technology, and collaboration, consider a career with us. We offer:
- A dynamic work culture that promotes innovation
- Opportunities for professional development 
- A chance to be part of a community that is shaping the future of AI

Explore current job openings on our [Jobs Page](#).

---

## Join Us Today
Are you ready to be part of the AI revolution? Sign up now to start collaborating, innovating, and building on Hugging Face.

[Explore AI Apps](#) | [Sign Up](#)

#### Connect With Us
- [GitHub](#)
- [Twitter](#)
- [LinkedIn](#)
- [Discord](#)

---

**Hugging Face – Your Partner in Building the Future of AI.**

#### Translate brochure to another language

In [11]:
translation_sys_prompt = "You are a language translator who is very good at translating business documents from \
English to any language. You preserve the formatting, tone and facts contained in the document."

def translate_brochure(brochure, language):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translation_sys_prompt},
            {"role": "user", "content": f"Translate the following document into {language}: {brochure}"}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [14]:
translate_brochure(brochure_text, "Spanish")

# Folleto de Hugging Face

## Bienvenido a Hugging Face
**La Comunidad de IA que Está Construyendo el Futuro**  
Hugging Face es una plataforma dinámica a la vanguardia de la comunidad de aprendizaje automático, promoviendo la colaboración en modelos, conjuntos de datos y aplicaciones innovadoras. Únete a nosotros para explorar más de **1 millón de modelos** y participar en una comunidad apasionada dedicada a acelerar el desarrollo de la IA.

---

## Lo Que Ofrecemos

### Colabora e Innova
Hugging Face es la plataforma preferida para entusiastas y profesionales del aprendizaje automático.  
- **Modelos**: Navega a través de una vasta colección de los últimos modelos de IA.
- **Conjuntos de datos**: Accede a más de **250,000 conjuntos de datos** diseñados para diversas tareas de aprendizaje automático.
- **Espacios**: Crea y comparte aplicaciones sin esfuerzo.

### Tecnología de Punta
Utiliza nuestro **stack de código abierto de HF** para mejorar tus proyectos en modalidades de texto, imagen, video, audio e incluso 3D. Aprovecha nuestras bibliotecas de última generación como:
- **Transformers:** Aprendizaje automático avanzado para PyTorch, TensorFlow y JAX.
- **Diffusers:** Modelos de difusión líderes.
- **Tokenizers:** Tokenizadores rápidos optimizados para investigación y producción.

---

## Nuestros Clientes
Con más de **50,000 organizaciones** utilizando nuestra plataforma, empoderamos tanto a pequeñas startups como a empresas de Fortune 500, incluyendo:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

Nuestras soluciones están dirigidas a empresas que buscan plataformas avanzadas con robusta seguridad y soporte dedicado, a partir de **$20/usuario/mes**.

---

## Cultura Empresarial
En Hugging Face, valoramos la colaboración, apertura e innovación. Nuestra cultura inclusiva fomenta la creatividad y anima a los miembros del equipo a contribuir y compartir sus ideas. Creemos en construir un ambiente de apoyo donde todos pueden aprender y crecer juntos, inmersos en la vibrante comunidad de IA.

---

## Carreras en Hugging Face
Siempre estamos buscando individuos talentosos para unirse a nuestro equipo. Si sientes pasión por el aprendizaje automático, la tecnología y la colaboración, considera una carrera con nosotros. Ofrecemos:
- Una cultura de trabajo dinámica que promueve la innovación
- Oportunidades para el desarrollo profesional 
- La oportunidad de ser parte de una comunidad que está moldeando el futuro de la IA

Explora las ofertas de trabajo actuales en nuestra [Página de Empleos](#).

---

## Únete a Nosotros Hoy
¿Estás listo para ser parte de la revolución de la IA? Regístrate ahora para comenzar a colaborar, innovar y construir en Hugging Face.

[Explorar Aplicaciones de IA](#) | [Regístrate](#)

#### Conéctate Con Nosotros
- [GitHub](#)
- [Twitter](#)
- [LinkedIn](#)
- [Discord](#)

---

**Hugging Face – Tu Socio en la Construcción del Futuro de la IA.**

## Stream output

Results stream back from OpenAI with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

## Chat UI

Build User Interface using the Gradio framework.

In [ ]:
import gradio as gr

In [ ]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result
        
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()